In [ ]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / 'src'

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    get_images_lists_from_path,
    get_images_lists_from_more_paths
)

from src.plots import (
    plot_observation
)

from src.loading import (
    load_images_from_paths
)

%load_ext autoreload
%autoreload 2

In [ ]:
type_to_use = 'flair'
seg_path = Path(os.getcwd()) / 'data_extracted' / 'seg'
input_path = Path(os.getcwd()) / 'data_extracted' / type_to_use

In [ ]:
images, segs = load_images_from_paths(input_path, seg_path)

# images_tensors = pad_sequence([torch.tensor(x) for x in images], batch_first=True)
# seg_tensors = pad_sequence([torch.tensor(x) for x in segs], batch_first=True)
# images_dataloader = DataLoader(images_tensors, batch_size=16, shuffle=True)

In [ ]:
# modelname = 'weights.pt'
#modelname = 'flair_model1.pt'
modelname = 'flair_model1_noshuffle.pt'
model = torch.load(Path(os.getcwd()) / 'models' / modelname)
model.eval()

In [ ]:
output_predictions.numpy().shape

In [ ]:
preprocess = transforms.Compose([
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor()
    ])

indexes_predict = np.arange(0, 2)

for i in indexes_predict:
    input_image = Image.fromarray(images[i])

    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) 

    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output

    # create a color pallette, selecting a color for each class
    palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
    colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
    colors = (colors % 255).numpy().astype("uint8")

    f, ax = plt.subplots(1, 2, figsize=(15, 4))
    ax[0].set_title('input image')
    ax[0].axis('off')
    ax[0].imshow(input_image)
    ax[1].set_title('segmented output')
    ax[1].axis('off')
    ax[1].imshow(torch.amax(output_predictions, 0))
    #plt.savefig("segmented_output.png", bbox_inches='tight')
    plt.show()